In [1]:
import os
import cv2
import json
from tqdm import tqdm
import shutil

# 경로 설정
train_data_path = '/data/ephemeral/home/data/train/DCM'
json_data_path = '/data/ephemeral/home/data/train/outputs_json'
crop_data_path_images = '/data/ephemeral/home/crop_data/DCM'
crop_data_path_json = '/data/ephemeral/home/crop_data/outputs_json'

# Crop 영역 정의 (예시)
crop_x, crop_y, crop_w, crop_h = 700, 1000, 700, 850  # x, y, width, height

# 크롭된 데이터 저장 디렉토리 생성
os.makedirs(crop_data_path_images, exist_ok=True)
os.makedirs(crop_data_path_json, exist_ok=True)

# tqdm을 사용하여 전체 이미지 수 계산
total_images = 0
for folder in os.listdir(train_data_path):
    folder_path = os.path.join(train_data_path, folder)
    if not os.path.isdir(folder_path):
        continue
    for img_file in os.listdir(folder_path):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.dcm')):
            total_images += 1

# DCM 파일을 처리하기 위한 함수 (DICOM 지원 필요 시 pydicom 사용)
def load_image(img_path):
    if img_path.lower().endswith('.dcm'):
        try:
            import pydicom
            ds = pydicom.dcmread(img_path)
            image = ds.pixel_array
            if len(image.shape) == 2:  # 그레이스케일
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            else:  # 이미 컬러 이미지인 경우
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            return image
        except Exception as e:
            print(f"Failed to load DCM image: {img_path}, Error: {e}")
            return None
    else:
        return cv2.imread(img_path)

# 모든 이미지 처리
with tqdm(total=total_images, desc="Processing Images") as pbar:
    for folder in os.listdir(train_data_path):
        folder_path = os.path.join(train_data_path, folder)
        if not os.path.isdir(folder_path):
            continue

        # 대응되는 JSON 폴더 경로
        json_folder_path = os.path.join(json_data_path, folder)
        if not os.path.exists(json_folder_path):
            print(f"JSON folder does not exist for folder: {folder}")
            continue

        # 크롭된 이미지와 JSON을 저장할 서브폴더 생성
        target_image_folder = os.path.join(crop_data_path_images, folder)
        target_json_folder = os.path.join(crop_data_path_json, folder)
        os.makedirs(target_image_folder, exist_ok=True)
        os.makedirs(target_json_folder, exist_ok=True)

        for img_file in os.listdir(folder_path):
            if not img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.dcm')):
                pbar.update(1)
                continue

            img_path = os.path.join(folder_path, img_file)
            image = load_image(img_path)
            if image is None:
                print(f"Failed to load image: {img_path}")
                pbar.update(1)
                continue

            # 이미지 크롭
            cropped_image = image[crop_y:crop_y + crop_h, crop_x:crop_x + crop_w]

            # 대응되는 JSON 파일 경로 (동일한 서브폴더 내)
            json_filename = os.path.splitext(img_file)[0] + '.json'
            json_path = os.path.join(json_folder_path, json_filename)

            if not os.path.exists(json_path):
                print(f"JSON file does not exist for image: {img_file}")
                pbar.update(1)
                continue

            # JSON 파일 로드
            try:
                with open(json_path, 'r') as f:
                    json_data = json.load(f)
            except Exception as e:
                print(f"Failed to load JSON file: {json_path}, Error: {e}")
                pbar.update(1)
                continue

            # JSON의 각 annotation의 points를 크롭에 맞게 조정
            for annotation in json_data.get("annotations", []):
                if "points" in annotation:
                    adjusted_points = []
                    for point in annotation["points"]:
                        adjusted_x = point[0] - crop_x
                        adjusted_y = point[1] - crop_y
                        # 크롭된 이미지 내에 있는지 확인
                        if 0 <= adjusted_x < crop_w and 0 <= adjusted_y < crop_h:
                            adjusted_points.append([adjusted_x, adjusted_y])
                        else:
                            # 포인트가 크롭 영역 밖이면 필요에 따라 처리 (예: 무시하거나 클리핑)
                            pass
                    if adjusted_points:
                        annotation["points"] = adjusted_points
                    else:
                        # 모든 포인트가 크롭 영역 밖이면 annotation을 제거하거나 처리
                        annotation["points"] = []

            # 크롭된 JSON 파일 저장
            cropped_json_path = os.path.join(target_json_folder, json_filename)
            try:
                with open(cropped_json_path, 'w') as f:
                    json.dump(json_data, f, indent=4)
            except Exception as e:
                print(f"Failed to save cropped JSON file: {cropped_json_path}, Error: {e}")

            # 크롭된 이미지 저장
            cropped_img_filename = img_file  # 동일한 파일 이름 사용
            cropped_img_path = os.path.join(target_image_folder, cropped_img_filename)
            try:
                if img_file.lower().endswith('.dcm'):
                    # DICOM 파일 저장은 복잡할 수 있으므로 여기서는 PNG로 저장
                    cropped_img_path = os.path.splitext(cropped_img_path)[0] + '.png'
                cv2.imwrite(cropped_img_path, cropped_image)
            except Exception as e:
                print(f"Failed to save cropped image: {cropped_img_path}, Error: {e}")

            pbar.update(1)

print("모든 이미지와 JSON 파일의 크롭이 완료되었습니다.")

Processing Images: 100%|██████████| 800/800 [01:17<00:00, 10.32it/s]

모든 이미지와 JSON 파일의 크롭이 완료되었습니다.


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


ModuleNotFoundError: No module named 'transform'

: 